# Model Comparisons in Demand Forecasting for E-Commerce

**Objective:** an ecommerce platform aimed to predict customer demand to optimize pricing and inventory management.

**Approach:** time series data of customer browsing behavior, product views, and historical sales were collected. Multiple models were used to capture seasonality and long-term dependencies.

**Outcome:** accurate demand forecasts helped the e-commerce platform optimize pricing, improve product recommendations, and efficiently manage inventory, resulting in increased sales and customer retention.

In [7]:
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from math import sqrt

/Users/stevesimonson/development/tutorials/PythonforDS/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load the data
data = pd.read_csv('../data_sets/demand.csv')

In [11]:
# Convert the 'week' column into datetime format
data['week'] = pd.to_datetime(data['week'], format='%d/%m/%y')
data.set_index('week', inplace=True)

# Aggregate the data by week
weekly_data = data['units_sold'].resample('W').sum()

In [13]:
# Train - Test Split
# train_data = weekly_data[:int(0.8*(len(weekly_data)))]
# test_data = weekly_data[int(0.8*(len(weekly_data))):]

from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(
    weekly_data,
    test_size=0.2,
    shuffle=False # Never shuffle time series data
)

## Models

### Holt-Winters model

In [14]:
# Fit
hw_model = ExponentialSmoothing(train_data, seasonal='add', seasonal_periods=52).fit()

# Predict
hw_predictions = hw_model.predict(start=test_data.index[0], end=test_data.index[-1])

### ARIMA

In [16]:
# Fit
arima_model = ARIMA(train_data, order=(1,0,0)).fit()
# using (1,0,0) from autocorrelation test. can also use grid search

# Predict
arima_predictions = arima_model.predict(start=test_data.index[0], end=test_data.index[-1])

### Facebook Prophet

In [19]:
# Prophet-specific data prep
prophet_data = weekly_data.reset_index()
prophet_data.columns = ['ds', 'y']

# Train Test spliit Prophet data
train_data_prophet = prophet_data[:int(0.8*(len(prophet_data)))]
test_data_prophet = prophet_data[int(0.8*(len(prophet_data))):]

# Fit
prophet_model = Prophet(yearly_seasonality=True)
prophet_model.fit(train_data_prophet)

# Predict
prophet_future = prophet_model.make_future_dataframe(periods=len(test_data_prophet))
prophet_predictions = prophet_model.predict(prophet_future)

20:47:46 - cmdstanpy - INFO - Chain [1] start processing
20:47:46 - cmdstanpy - INFO - Chain [1] done processing


## Model Comparison Results


|Model|RMSE|
|---|---|
|Holt's Winter Exponential Smoothing|9283|
